In [1]:
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SparkSession
from operator import add
import nltk
import json
from time import time
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')

In [2]:
_tstart_stack = []
def tic():
    _tstart_stack.append(time())
def toc(fmt="Elapsed: %s s"):
    print (fmt % (time() - _tstart_stack.pop()))

In [3]:
gram_groups = dict()
gram_groups['Adjectives'] = ['JJ', 'JJR', 'JJS']
gram_groups['Nouns'] = ['NN', 'NNS', 'NNP', 'NNPS']
gram_groups['Pronouns'] = ['PRP', 'PRP$']
gram_groups['Adverbs'] = ['RB', 'RBR', 'RBS']
gram_groups['Verb'] = ['VB', 'VBG', 'VBD', 'VBN', 'VBP', 'VBZ']

def get_split_body(rdd):
    body = rdd\
    .map(lambda line: json.loads(line[1])['body'].strip().split())\
    
    return body

def check_gram_grp(tag_tuple):
    word, tag = tag_tuple
    for supergroup, subgroups in gram_groups.items():
                if tag in subgroups:
                    return supergroup
    return None

def categorize_words(split_rdd):
    cat_words = split_rdd.flatMap(lambda word: nltk.pos_tag(word))\
    .map(lambda tupl: (check_gram_grp(tupl), 1))\
    .filter(lambda x: x[0] != None)
   
    return cat_words

def final_test(rdd):
    split = get_split_body(rdd)
    categorized = categorize_words(split)
    group_counts = categorized.reduceByKey(add).collect()
    return group_counts

In [5]:
# (8 cores, 16gb per machine) x 5 = 40 cores

# New API
spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.87:7077") \
        .appName("blameyben_lecture1_simple_example")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.shuffle.service.enabled", True)\
        .getOrCreate()
        #.config("spark.executor.cores",2)\
# Old API (RDD)
spark_context = spark_session.sparkContext

In [6]:
#confSpark = SparkConf().set("spark.driver.bindAddress", "localhost")
#sc = SparkContext("local[*]", "appname", conf = confSpark)

# Old API (RDD)
#spark_context = spark_session.sparkContext
# New API
#spark_session = SparkSession\
#        .builder\
#        .master("spark://192.168.2.119:7077") \
#        .appName("TestApp")\
#        .config("spark.dynamicAllocation.enabled", False)\
#        .config("spark.shuffle.service.enabled", False)\
#        .config("spark.cores.max",4)\
#        .getOrCreate()


# Old API (RDD)
#spark_context = spark_session.sparkContext

#spark_context.setLogLevel("INFO")

read a bunch of rdds form htfs into one rdd; the rdd should be stors in the node in the worker (not on the master)
rdd(1:m), rdd(m:2m), rddtot(1:2m)
run our example on rdd(1:2m), with different cpu (strong)
run rdd(1:m) on 1 cpu record time
and rdd(1:m) and rdd(m:2m) on 2 cpu record time
and store the time (weak)



In [7]:

rdd = spark_context.newAPIHadoopFile(
    'hdfs://localhost:9000/user/ubuntu/RC_2008-02',
    'org.apache.hadoop.mapreduce.lib.input.TextInputFormat',
    'org.apache.hadoop.io.LongWritable',
    'org.apache.hadoop.io.Text'
)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.newAPIHadoopFile.
: org.apache.hadoop.mapreduce.lib.input.InvalidInputException: Input path does not exist: hdfs://localhost:9000/user/ubuntu/RC_2008-02
	at org.apache.hadoop.mapreduce.lib.input.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:323)
	at org.apache.hadoop.mapreduce.lib.input.FileInputFormat.listStatus(FileInputFormat.java:265)
	at org.apache.hadoop.mapreduce.lib.input.FileInputFormat.getSplits(FileInputFormat.java:387)
	at org.apache.spark.rdd.NewHadoopRDD.getPartitions(NewHadoopRDD.scala:130)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:253)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:251)
	at scala.Option.getOrElse(Option.scala:121)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:251)
	at org.apache.spark.rdd.MapPartitionsRDD.getPartitions(MapPartitionsRDD.scala:49)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RD

#### RDD  info

In [ ]:
np = rdd.getNumPartitions()
n = rdd.count()
print(n,np)

In [ ]:
#x = rdd.take(1)[0][1]
#json.loads(x)

#### Create an rdd with half size

In [ ]:
rdd_half = rdd.sample(False, 0.5)

#### Test with full data

In [ ]:
tic()
final_test(rdd)
toc()

#### Test with half data

In [ ]:
tic()
final_test(rdd_half)
toc()